<a href="https://colab.research.google.com/github/sarjakpatel/SJSU/blob/main/CMPE%20258/Assignment%202/TensorOperationsUsingTensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMPE 258 Assignment 2 : Tensor Operations using TensorFlow

In [1]:
import tensorflow as tf

**Basic Operations on tf.Tensor object:**

Addition:

In [22]:
print(tf.add(10, 11))
print(tf.add([5, 6], [10, 11]))

tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor([15 17], shape=(2,), dtype=int32)


In [23]:
print(tf.square(8))
print(tf.reduce_sum([4, 5, 6]))

tf.Tensor(64, shape=(), dtype=int32)
tf.Tensor(15, shape=(), dtype=int32)


operator overloading:

In [24]:
print(tf.square(5) + tf.square(6)) #

tf.Tensor(61, shape=(), dtype=int32)


Matrix Multiplication:

In [25]:
matrix = tf.matmul([[1]], [[2, 3]])
print(matrix)
print(matrix.shape)
print(matrix.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


**Conversion between NumPy and Tensorflow**

In [26]:
import numpy as np

In [27]:
numpyArray = np.ones([2, 2])

In [28]:
tensor = tf.multiply(numpyArray, 100)
print(tensor)

tf.Tensor(
[[100. 100.]
 [100. 100.]], shape=(2, 2), dtype=float64)


In [30]:
print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

The .numpy() method explicitly converts a Tensor to a numpy array
[[100. 100.]
 [100. 100.]]


**GPU acceleration**

In [31]:
x = tf.random.uniform([2, 2])

print("Is there a GPU available: "),
print(tf.config.list_physical_devices("GPU"))

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

Is there a GPU available: 
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is the Tensor on GPU #0:  
True


In [33]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(5):
    tf.matmul(x, x)

  result = time.time()-start

  print("5 loops: {:0.2f}ms".format(1000*result))

# Force execution on CPU
print("Matrix Multiplication On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([5000, 5000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# Force execution on GPU #0 if available
if tf.config.list_physical_devices("GPU"):
  print("Matrix Multiplication On GPU:")
  with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
    x = tf.random.uniform([5000, 5000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

Matrix Multiplication On CPU:
5 loops: 17393.15ms
Matrix Multiplication On GPU:
5 loops: 1.19ms


In [34]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Create a CSV file
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

In [35]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

In [36]:
print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x)

print('\nElements in ds_file:')
for x in ds_file:
  print(x)

Elements of ds_tensors:
tf.Tensor([4 1], shape=(2,), dtype=int32)
tf.Tensor([ 9 25], shape=(2,), dtype=int32)
tf.Tensor([16 36], shape=(2,), dtype=int32)

Elements in ds_file:
tf.Tensor([b'Line 1' b'Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'Line 3' b'  '], shape=(2,), dtype=string)


**Einsum Operations:**

In [38]:
m0 = tf.random.normal(shape=[2, 4])
m1 = tf.random.normal(shape=[4, 5])
e = tf.einsum('ij,jk->ik', m0, m1)
# output[i,k] = sum_j m0[i,j] * m1[j, k]
print(e.shape)


(2, 5)


In [39]:
e = tf.einsum('ij,jk', m0, m1)  # output[i,k] = sum_j m0[i,j] * m1[j, k]
print(e.shape)

(2, 5)


In [40]:
u = tf.random.normal(shape=[5])
v = tf.random.normal(shape=[5])
e = tf.einsum('i,i->', u, v)  # output = sum_i u[i]*v[i]
print(e.shape)

()


In [41]:
u = tf.random.normal(shape=[3])
v = tf.random.normal(shape=[5])
e = tf.einsum('i,j->ij', u, v)  # output[i,j] = u[i]*v[j]
print(e.shape)

(3, 5)


In [42]:
m = tf.ones(2,3)
e = tf.einsum('ij->ji', m0)  # output[j,i] = m0[i,j]
print(e.shape)

(4, 2)


In [43]:
m = tf.reshape(tf.range(9), [3,3])
diag = tf.einsum('ii->i', m)
print(diag.shape)

(3,)


In [44]:
trace = tf.einsum('ii', m)  # output[j,i] = trace(m) = sum_i m[i, i]
assert trace == sum(diag)
print(trace.shape)

()


In [45]:
s = tf.random.normal(shape=[7,5,3])
t = tf.random.normal(shape=[7,3,2])
e = tf.einsum('bij,bjk->bik', s, t)
# output[a,i,k] = sum_j s[a,i,j] * t[a, j, k]
print(e.shape)

(7, 5, 2)


In [46]:
s = tf.random.normal(shape=[11, 7, 5, 3])
t = tf.random.normal(shape=[11, 7, 3, 2])
e =  tf.einsum('...ij,...jk->...ik', s, t)
print(e.shape)

(11, 7, 5, 2)
